In [1]:
import pandas as pd
import numpy as np

In [1]:
class Wind:
    """ relative winddirection for now as input from 0-180 on the boat looking in the heading direction
    H = [m] height of the vessel m
    B = [m] width of the vessel 
    L = [m] length of the vessel
    rel_winddir = [deg] relative winddirection from 0-180 from looking at winddirection and heading, 0 is from the head, 180 from stern ! manually
    loaded_depth = [m] depth of vessel loaded
    unloaded_depth = [m] depth of vessel unloaded
    load = 'loaded'/'unloaded', for depth and cd
    V_g = [m/s] speed over ground 
    heading = [deg] direction 0 degrees is north , 45 degrees is northeast. 
    winddirection = [deg] geographic wind direction
    U_wind = [m/s] windspeed
    rho_air = [kg/m3] density of air
    rho_water = [kg/m3] density of water
    """
    def __init__(
        #""" relative winddirection for now as input from 0-180 on the boat looking in the heading direction
        #"""
        self,
        B, 
        L,
        H ,
        rel_winddir,
        loaded_depth,
        unloaded_depth,
        load = 'loaded',
     
        V_g = 0.3,
        heading = 40,# degrees (NE)
        winddirection = 60,# degrees (not relative but real)
        U_wind = 2,
        rho_air = 1.225,
        rho_water = 1025
        
        
        
    ):
        self.B = B
        self.L = L
        self.H = H
        self.load = load
        self.V_g = V_g
        self.heading = heading
        self.winddirection = winddirection
        self.U_wind = U_wind
        self.rho_air = rho_air
        self.rel_winddir = rel_winddir
        self.loaded_depth = loaded_depth
        self.unloaded_depth = unloaded_depth
        self.rho_water = rho_water
       
    
    def calculate_rel_winddir (self):
        #phi_w_ref = angle of attack  
        #(https://sciendo.com/pdf/10.2478/ntpe-2018-0013)
        # pffff still not found the right relations for angle, winddirection. 
            
        self.phi_w_ref = self.winddirection - self.heading  
        return self.phi_w_ref
    
    def calculate_C_d(self):
        # for a tanker ( ITTC: ) 
        # cda = -cx (cx given for ships)
        
        cd=[[0,0.86,0.96],[10,0.76,0.93],[20,0.62,0.85],
        [30,0.45,0.73],[40,0.32,0.62],[50,0.21,0.47],
        [60,0.13,0.34],[70,0.06,0.17],[80,0.04,0.06],
        [90,-0.02,-0.05],[100,-0.08,-0.14],[110,-0.19,-0.22],
        [120,-0.29,-0.29],[130,-0.38,-0.40],[140,-0.47,-0.53],
        [150,-0.56,-0.66],[160,-0.61,-0.75],[170,-0.66,-0.79],
        [180,-0.63,-0.77]]
    
        df=pd.DataFrame(cd,columns=['Relative wind direction','loaded','unloaded'])
    
        if self.load == 'loaded':
            self.C_d = df['loaded'].where(df['Relative wind direction'] == self.rel_winddir).dropna().values[0]
    
        elif self.load == 'unloaded':
            self.C_d = df['unloaded'].where(df['Relative wind direction'] == self.rel_winddir).dropna().values[0]
    
        return self.C_d
    
    def calculate_Cd_0(self):
        if self.load == 'loaded':
            self.Cd_0 = 0.86                            
        elif self.load == 'unloaded':
            self.Cd_0 = 0.96
        return self.Cd_0 
    
    def calculate_H_above_water(self):
        if self.load == 'loaded':
            self.H_above = self.H - self.loaded_depth 
        elif self.load == 'unloaded':
            self.H_above = self.H - self.unloaded_depth
        return self.H_above
    
    def calculate_A_xv(self):
        #cubed barge assumed 
        angle=0
        if self.rel_winddir <91:
            self.A_xv = self.calculate_H_above_water() * (self.B * np.cos(self.rel_winddir*np.pi/180) + self.L * np.sin(self.rel_winddir*np.pi/180))
            self.A_ydir = self.calculate_H_above_water() * self.L * np.sin(self.rel_winddir*np.pi/180)
        elif self.rel_winddir > 90:
            angle = 90-(self.rel_winddir -90) # symmetric shape front and back. 
            self.A_xv = self.calculate_H_above_water() * (self.B * np.cos(angle*np.pi/180) + self.L * np.sin(angle*np.pi/180))
            self.A_ydir = self.calculate_H_above_water() * self.L * np.sin(angle*np.pi/180)
        return self.A_xv , angle , self.A_ydir
    
    def calculate_wind_resistance(self):
        self.R_wind = 0.5 * self.calculate_C_d() * self.rho_air * self.calculate_A_xv()[0] * self.U_wind **2 - 0.5 * self.rho_air * self.calculate_Cd_0() * self.calculate_A_xv()[0] * self.V_g **2
    # negative value is in the Cd  value. so negative winddirection gives negative value because of cd. 
        return self.R_wind , 'N'
    
    # for the passive rudder: 
    # 1. calculate the moment caused by wind and distance to centre of gravity # assume stays constant in middle
    # 2. moment for counteraction is same, with distance to centre of gravity (assume stays constant) force fN,delta_r can be calculated. 
    # 3. depends on angle delta_R the force as well as the moment, look at different angles so that smallest angle with R is chosen
    # 4. print angle so that moment is the same.
    # 5. calculate Rrx with Rrx = abs(Fn*sin(delta_r))
    
    def calculate_moment_wind(self):
        self.wind_ydir = 0.5 * self.rho_air * self.calculate_A_xv()[2] * self.U_wind **2 * 1 
        self.mom_wind = self.wind_ydir * self.L
        # cd assumed 1, not sure if same cd as 90 deg as that is very small number.
        return self.wind_ydir , self.mom_wind
        
    def calculate_FN_cosdr(self):
        self.dis_to_cog = self.L / 2
        lamda = 600 # first assumption https://www.marinesite.info/2021/05/aspect-ratio-force-acting-on-rudder.html
        A_R = 1/60 * self.B * self.L #https://www.marinesite.info/2021/05/aspect-ratio-force-acting-on-rudder.html
        self.dr = 7
        FN =   0.5 * self.rho_water * ( 6.13 * lamda/(lamda+2.25))* A_R * self.V_g **2 * np.sin(self.dr / 2 * np.pi/180)      
        Rry = np.cos(self.dr * np.pi/180) *  FN
        Rrx = np.abs(FN*np.sin(self.dr * np.pi/180)) 
        Mrz = Rry * self.dis_to_cog    
#         #alpha_r = delta_r/2 assumption
#         #ignored hull factor ay on rudder force for now
        
        return Rry, Rrx, Mrz, self.dr , FN 
    
    def calculate_angle(self):
        lamda = 600 
        A_R = 1/60 * self.B * self.L
        dr = np.arange(1,41,1)
        windmoment = self.calculate_moment_wind()[1]
        
        for i in range(41):
            FN =   0.5 * self.rho_water * ( 6.13 * lamda/(lamda+2.25))* A_R * self.V_g **2 * np.sin(dr[i] / 2 * np.pi/180)      
            Rry = np.cos(dr[i] * np.pi/180) *  FN
            Mry = Rry * self.L/2
            if  Mry > windmoment:
                
                angle = dr[i]
                
                Rrx = np.abs(FN*np.sin(dr[i] * np.pi/180))
                return angle, windmoment, Mry, Rrx
            
        else:
            print('too large angle')
                
        
    


In [2]:
p =  Wind(B = 11,L = 60,rel_winddir=20,load = 'loaded',V_g = 0.3,heading = 40,winddirection = 60,U_wind = 15,rho_air = 1.225, H = 4, loaded_depth = 3.45, unloaded_depth = 2)


In [3]:
#print(p.calculate_FN_cosdr()[1],'H_above')
print(p.calculate_A_xv(), 'Axv')
print(p.calculate_C_d(), 'C_d')
print(p.calculate_rel_winddir(), 'rel_winddir')
print(p.calculate_moment_wind(), 'windmoment')

NameError: name 'np' is not defined

In [ ]:
p.calculate_wind_resistance()

In [ ]:
#print(p.calculate_FN_cosdr()[1],'H_above')
print(p.calculate_A_xv(), 'Axv')
print(p.calculate_C_d(), 'C_d')
print(p.calculate_rel_winddir(), 'rel_winddir')
print(p.calculate_moment_wind()[1], 'windmoment')
print(p.calculate_FN_cosdr()[2], 'countermoment')
print(p.calculate_FN_cosdr())
print(p.calculate_FN_cosdr()[1], 'extra resistance')
# dit is niet die counter, maar de extra resistance door de counter. 